# Data Extraction Overview

This notebook begins our data science project by extracting data from an SQLite database, setting the stage for subsequent data inspection and cleaning phases. We utilize two main functions to facilitate this process:

- **`get_table_as_df`**: This function retrieves a specific table from the database connection and returns it as a Pandas DataFrame, enabling straightforward data manipulation.
- **`get_all_dataframes_from_database`**: This function automatically fetches all tables from the specified SQLite database and converts them into DataFrames.

For this project, we access the database located at `"dev/cademycode.db"` and load each table into separate variables for detailed analysis. This initial step is crucial as it prepares our dataset for further exploration and preprocessing tasks.


In [35]:
import sqlite3
import pandas as pd
from collections import namedtuple
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

# Uses sqlite3 connection and table name to get dataframe 
def get_table_as_df(connection: sqlite3.Connection, table: str) -> pd.DataFrame:
    try:
        query = f"SELECT * FROM {table}"
        dataframe = pd.read_sql_query(query, connection)
    except sqlite3.Error as error:
        print(f"Error while connecting to SQLite database {error}", error)
        raise error
    except Exception as e:
        print(f"Error while reading data from the database: {e}")
        raise e
    return dataframe

# Gets all tables as Pandas DataFrame from a database
def get_all_dataframes_from_database(database: str) -> namedtuple:
    try: 
        with sqlite3.connect(database) as connection:
            dataframes = []
            table_names = [x[0] for x in connection.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]
            if not table_names:
                raise ValueError("No tables found")
            DataF = namedtuple("DataF", table_names)
            for table_name in table_names:
                dataframes.append(get_table_as_df(connection, table_name))
            dataframe_tuple = DataF(*dataframes)
            return dataframe_tuple
    except sqlite3.Error as error:
        print(f"Error while connecting to SQLite database {error}")
        raise error

In [36]:
# Getting our dataframes from cademycode.db into variable for future data inspecting and cleaning
cmc_data = get_all_dataframes_from_database("dev/cademycode.db")

# At first let's drop duplicate values and check dtypes
for i, cmc in enumerate(cmc_data):
    print(cmc_data._fields[i])
    cmc.drop_duplicates(inplace=True)
    print(cmc.info())

cademycode_students
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB
None
cademycode_courses
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-

## Observations of Column Data

Although column names are consistent, the data types do not always match their values. Below are the columns that need adjustments:

- **cademycode_students**:
    - Convert `dob` from `object` to `datetime64`
    - Convert `sex` from `object` to `category`
    - Convert `job_id` from `object` to `int64`
    - Convert `num_course_taken` from `object` to `int64`
    - Convert `current_career_path_id` from `object` to `int64`
    - Convert `time_spent_hrs` from `object` to `float64`

Other tables has appropriate data types and there is no need to change anything.
Before converting data types we will need to deal with null values.


In [37]:
# cadymycode_students table
cmc_data.cademycode_students

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64
...,...,...,...,...,...,...,...,...,...
4995,4996,Quentin van Harn,1967-07-07,N,"{""mailing_address"": ""591 Blue Berry, Coulee, I...",5.0,5.0,2.0,13.82
4996,4997,Alejandro van der Sluijs,1964-11-03,M,"{""mailing_address"": ""30 Iron Divide, Pewaukee ...",4.0,13.0,1.0,7.86
4997,4998,Brock Mckenzie,2004-11-25,M,"{""mailing_address"": ""684 Rustic Rest Avenue, C...",8.0,10.0,3.0,12.1
4998,4999,Donnetta Dillard,1943-02-12,N,"{""mailing_address"": ""900 Indian Oval, Euclid, ...",3.0,6.0,5.0,14.86


## Handling null values
But before we start to convert our data types, first we need to deal with null values. Above we can see in what columns we have missing data.But for clarity I've created table with percent of missing data in each column. 

In [52]:
cmc_students_md = cmc_data.cademycode_students.isnull().sum()
percent_md = (cmc_students_md / len(cmc_data.cademycode_students)) * 100
missing_data_summary = pd.DataFrame({
    'Percentage': percent_md,
    'Missing Values': cmc_students_md
})
missing_data_summary

,Percentage,Missing Values
uuid,0.00,0
name,0.00,0
dob,0.00,0
sex,0.00,0
contact_info,0.00,0
job_id,0.10,5
num_course_taken,5.02,251
current_career_path_id,9.42,471
time_spent_hrs,9.42,471


### Analysing the `job_id` column 
Let's see why we are missing data in our `job_id` column.

In [61]:
# Converting dob column to datetime dtype
cmc_data.cademycode_students['dob'] = pd.to_datetime(cmc_data.cademycode_students['dob'])
# We saw that we are missing 5 values in job_id column and its missing only where dob is 2002.
cmc_data.cademycode_students[(cmc_data.cademycode_students['dob'].dt.year == 2002)]

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
58,59,Tyron van Riemsdijk,2002-07-29,M,"{""mailing_address"": ""899 E Orchard, Estelline,...",8.0,12.0,8.0,6.17
132,133,Henry Estes,2002-07-25,F,"{""mailing_address"": ""357 Fallen Shadow, Seldov...",8.0,2.0,5.0,17.96
162,163,Glen Riley,2002-08-22,M,"{""mailing_address"": ""P.O. Box 37267, Cornlea v...",None,8.0,3.0,5.7
204,205,Kasey Conley,2002-06-02,N,"{""mailing_address"": ""172 S Mall, Streator, Nor...",8.0,5.0,5.0,16.95
264,265,Janna Abbott,2002-12-26,N,"{""mailing_address"": ""233 Stony Grove Divide, N...",8.0,6.0,2.0,4.39
...,...,...,...,...,...,...,...,...,...
4454,4455,Trent Finch,2002-01-04,M,"{""mailing_address"": ""841 Green Stream, Ostrand...",8.0,4.0,3.0,10.49
4473,4474,Miquel Bonilla,2002-01-28,M,"{""mailing_address"": ""895 Center Landing, Rutle...",8.0,4.0,1.0,9.55
4548,4549,Miss Black,2002-08-03,F,"{""mailing_address"": ""26 Dusty Log, Diamond vil...",8.0,5.0,6.0,8.81
4585,4586,Loni Hogan,2002-11-26,F,"{""mailing_address"": ""943 Silver Motorway, Reev...",8.0,5.0,7.0,10.11


### Conclusion for the `job_id` Column

As observed, there is no specific dependency between the `dob` and `job_id` columns. It is noted that other students born in 2002 may have jobs; however, it's also possible that due to their age, they are still studying and not working. Therefore, I will fill the missing values in the `job_id` column with the placeholder 'unknown'.


In [64]:
# Filling missing values with the placeholder
cmc_data.cademycode_students['job_id'].fillna('unknown', inplace=True)
cmc_data.cademycode_students[cmc_data.cademycode_students['job_id'] == 'unknown']

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
162,163,Glen Riley,2002-08-22,M,"{""mailing_address"": ""P.O. Box 37267, Cornlea v...",unknown,8.0,3.0,5.7
757,758,Mercedez Vorberg,2002-03-25,F,"{""mailing_address"": ""284 Cedar Seventh, Virden...",unknown,15.0,4.0,4.14
854,855,Kurt Ho,2002-05-29,M,"{""mailing_address"": ""P.O. Box 27254, Olin, New...",unknown,0.0,8.0,23.72
1029,1030,Penny Gaines,2002-03-01,N,"{""mailing_address"": ""138 Misty Vale, Stockton ...",unknown,15.0,4.0,16.25
1542,1543,Frederick Reilly,2002-11-13,M,"{""mailing_address"": ""P.O. Box 40769, Quakervil...",unknown,7.0,9.0,21.32
